In [226]:
import numpy as np
import random
import time
import math

In [227]:
data_path = "./data/"

files = ["brock200_1", "brock200_2", "brock200_3", "brock200_4", "c-fat200-1", "c-fat200-2", "c-fat200-5", "c-fat500-1", "c-fat500-10", "c-fat500-2", "c-fat500-5", "C125.9", "gen200_p0.9_44", "gen200_p0.9_55",  "johnson8-2-4",  "johnson8-4-4", "johnson16-2-4", "hamming6-2", "hamming6-4", "hamming8-2", "hamming8-4", "keller4", "MANN_a9", "MANN_a27", "MANN_a45", "p_hat300-1", "p_hat300-2", "p_hat300-3", "san200_0.7_1", "san200_0.7_2", "san200_0.9_1", "san200_0.9_2", "san200_0.9_3", "sanr200_0.7"]

In [228]:
data = {} 
for file in files:
    data[file] = {"vertex_num": None, "edge_num": None, "edges": {}}
    with open(data_path + file + ".clq", "r") as f:
        for row in f:
            if row[0].startswith('c'):
                continue
            elif row[0].startswith('p'):
                data[file]["vertex_num"], data[file]["edge_num"] = int(row.split()[-2]), int(row.split()[-1])
            else:
                vertex1, vertex2 = int(row.split()[-2]) - 1, int(row.split()[-1]) - 1

                if vertex1 not in data[file]["edges"].keys():
                    data[file]["edges"][vertex1] = {vertex2}
                elif vertex2 not in data[file]["edges"][vertex1]:
                    data[file]["edges"][vertex1].add(vertex2)

                if vertex2 not in data[file]["edges"].keys():
                    data[file]["edges"][vertex2] = {vertex1}
                elif vertex1 not in data[file]["edges"][vertex2]:
                    data[file]["edges"][vertex2].add(vertex1)
        data[file]["edges"] = dict(sorted(data[file]["edges"].items()))

In [229]:
for dataset in data.keys():
    data[dataset]["weights"] = {}
    data[dataset]["weights"] = [math.ceil(10*i / data[dataset]["vertex_num"]) * 0.1 for i in range(1, data[dataset]["vertex_num"]+1)] 

In [230]:
def find_clique(edges: dict, weights: dict):
    weighted_clique = []
    weight = 0
    original_candidates = set(edges.keys())
    original_candidates_degrees = [len(edges[vertex]) for vertex in original_candidates]
    probability_weights = [10*original_candidates_degrees[vertex] + 1*weights[vertex]  for vertex in edges.keys()]
    candidates = original_candidates.copy()
    while len(candidates) != 0:
        candidates_probability_weights = [probability_weights[i] for i in candidates]
        vertex = random.choices(population=list(candidates), weights=candidates_probability_weights, k=1)[0]
            
        weighted_clique.append(vertex)
        weight += weights[vertex]
            
        candidates = candidates.intersection(edges[vertex])
        candidates = candidates.intersection(edges[vertex])
    return weight, weighted_clique

In [231]:
def randomized_greedy_max_clique(edges: dict, weights: dict) -> list:
    attempts = 0
    best_weighted_clique = []
    best_weight = 0
    while attempts < 10000:
        weight, weighted_clique = find_clique(edges, weights)
        if weight > best_weight:
            best_weighted_clique = weighted_clique.copy()
            best_weight = weight
            attempts = 0
        else:
            attempts += 1

    return best_weight, best_weighted_clique

In [232]:
for dataset in data.keys():
        for i in range(10):
                time_start = time.perf_counter()
                all_vertices = set(range(len(data[dataset]["edges"].keys())))
                data[dataset]["additions"] = {}
                for vertex in data[dataset]["edges"].keys():
                        data[dataset]["additions"][vertex] = all_vertices - data[dataset]["edges"][vertex] - set([vertex])
                data[dataset]["best_weight"], data[dataset]["best_weighted_clique"] = randomized_greedy_max_clique(data[dataset]["additions"], data[dataset]["weights"])
        time_working = time.perf_counter() - time_start
        print(f'Наименование датасета: {dataset}, Время работы алгоритма: {time_working}, Лучший вес: {data[dataset]["best_weight"]}, Лучшая клика: {data[dataset]["best_weighted_clique"]}')

Наименование датасета: brock200_1, Время работы алгоритма: 1.7803476000008231, Лучший вес: 5.2, Лучшая клика: [180, 191, 99, 187, 139, 199]
Наименование датасета: brock200_2, Время работы алгоритма: 2.458053800000016, Лучший вес: 7.7, Лучшая клика: [190, 123, 191, 173, 166, 179, 142, 198, 82]
Наименование датасета: brock200_3, Время работы алгоритма: 2.551220000000285, Лучший вес: 5.900000000000001, Лучшая клика: [116, 187, 130, 37, 126, 148, 190, 176]
Наименование датасета: brock200_4, Время работы алгоритма: 3.6916957000003094, Лучший вес: 5.800000000000001, Лучшая клика: [186, 139, 154, 176, 109, 179, 175]
Наименование датасета: c-fat200-1, Время работы алгоритма: 3.8106207000000722, Лучший вес: 13.100000000000001, Лучшая клика: [148, 171, 152, 128, 64, 169, 108, 179, 173, 187, 140, 167, 197, 191, 89, 45, 121]
Наименование датасета: c-fat200-2, Время работы алгоритма: 4.43862689999969, Лучший вес: 7.6000000000000005, Лучшая клика: [180, 186, 166, 138, 176, 170, 146, 70, 190]
Наимено